<a href="https://colab.research.google.com/github/sakkarin31/musicrecomment/blob/main/testbesttofind.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from scipy.spatial import KDTree

In [6]:
# โหลด Dataset
df = pd.read_csv("tracks_features.csv")

# ดูตัวอย่างข้อมูล
df.head()

,id,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,danceability,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,release_date
0,7lmeHLHBe4nmXzuXc0HDjk,Testify,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],1,1,False,0.470,...,0.0727,0.02610,0.000011,0.3560,0.503,117.906,210133,4.0,1999,1999-11-02
1,1wsRitfRRtWyEapl0q22o8,Guerrilla Radio,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],2,1,True,0.599,...,0.1880,0.01290,0.000071,0.1550,0.489,103.680,206200,4.0,1999,1999-11-02
2,1hR0fIFK2qRG3f3RF70pb7,Calm Like a Bomb,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],3,1,False,0.315,...,0.4830,0.02340,0.000002,0.1220,0.370,149.749,298893,4.0,1999,1999-11-02
3,2lbASgTSoDO7MTuLAXlTW0,Mic Check,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],4,1,True,0.440,...,0.2370,0.16300,0.000004,0.1210,0.574,96.752,213640,4.0,1999,1999-11-02
4,1MQTmpYOZ6fcMQc56Hdo7T,Sleep Now In the Fire,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],5,1,False,0.426,...,0.0701,0.00162,0.105000,0.0789,0.539,127.059,205600,4.0,1999,1999-11-02


In [7]:
# เลือกเฉพาะ Feature ที่ใช้เทรนโมเดล
features = ['danceability', 'energy', 'key', 'loudness', 'speechiness',
            'acousticness', 'instrumentalness', 'valence', 'tempo']

# ลบค่าที่เป็น NaN
df_clean = df.dropna(subset=features)

# แปลงข้อมูลเป็น numpy array
X = df_clean[features].values

# Normalize ข้อมูล
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# แบ่งข้อมูล train/test
X_train, X_test = train_test_split(X_scaled, test_size=0.2, random_state=42)

print("ขนาดของข้อมูล Train:", X_train.shape)
print("ขนาดของข้อมูล Test:", X_test.shape)


ขนาดของข้อมูล Train: (963220, 9)
ขนาดของข้อมูล Test: (240805, 9)


In [8]:
# กำหนดขนาด Input
input_dim = X_train.shape[1]

# Encoder
input_layer = Input(shape=(input_dim,))
encoded = Dense(64, activation='relu')(input_layer)
encoded = Dense(32, activation='relu')(encoded)
encoded = Dense(16, activation='relu')(encoded)

# Decoder
decoded = Dense(32, activation='relu')(encoded)
decoded = Dense(64, activation='relu')(decoded)
decoded = Dense(input_dim, activation='linear')(decoded)

# สร้างโมเดล Autoencoder
autoencoder = Model(input_layer, decoded)

autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

# ดูโครงสร้างโมเดล
autoencoder.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 9)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │             640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 16)                  │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 32)                  │             544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 64)                  │           2,112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 9)                   │             585 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,489 (25.35 KB)

 Trainable params: 6,489 (25.35 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
# เทรนโมเดล
history = autoencoder.fit(X_train, X_train, epochs=50, batch_size=32, shuffle=True, validation_data=(X_test, X_test))

Epoch 1/50
30101/30101 ━━━━━━━━━━━━━━━━━━━━ 88s 3ms/step - loss: 0.0189 - val_loss: 9.2264e-05
Epoch 2/50
30101/30101 ━━━━━━━━━━━━━━━━━━━━ 144s 3ms/step - loss: 1.5678e-04 - val_loss: 6.8264e-05
Epoch 3/50
30101/30101 ━━━━━━━━━━━━━━━━━━━━ 136s 3ms/step - loss: 1.4106e-04 - val_loss: 3.9595e-05
Epoch 4/50
30101/30101 ━━━━━━━━━━━━━━━━━━━━ 83s 3ms/step - loss: 1.2183e-04 - val_loss: 1.3335e-04
Epoch 5/50
30101/30101 ━━━━━━━━━━━━━━━━━━━━ 90s 3ms/step - loss: 1.0647e-04 - val_loss: 1.9499e-05
Epoch 6/50
30101/30101 ━━━━━━━━━━━━━━━━━━━━ 89s 3ms/step - loss: 1.0451e-04 - val_loss: 5.4980e-05
Epoch 7/50
30101/30101 ━━━━━━━━━━━━━━━━━━━━ 142s 3ms/step - loss: 1.0104e-04 - val_loss: 1.4084e-04
Epoch 8/50
30101/30101 ━━━━━━━━━━━━━━━━━━━━ 88s 3ms/step - loss: 9.9191e-05 - val_loss: 1.5995e-04
Epoch 9/50
30101/30101 ━━━━━━━━━━━━━━━━━━━━ 89s 3ms/step - loss: 9.1050e-05 - val_loss: 1.4151e-04
Epoch 10/50
30101/30101 ━━━━━━━━━━━━━━━━━━━━ 135s 3ms/step - loss: 8.3253e-05 - val_loss: 9.9088e-05
Epoch 11/

In [10]:
# สร้าง Encoder Model
encoder = Model(input_layer, encoded)

# แปลงเพลงเป็นเวกเตอร์
X_encoded = encoder.predict(X_scaled)

# เพิ่มคอลัมน์เวกเตอร์ลงใน DataFrame
df_clean["vector"] = list(X_encoded)

print("ตัวอย่างเวกเตอร์ของเพลงแรก:", df_clean["vector"].iloc[0])


37626/37626 ━━━━━━━━━━━━━━━━━━━━ 50s 1ms/step
ตัวอย่างเวกเตอร์ของเพลงแรก: [1.2540829  1.1643057  0.8006325  0.         0.7784927  1.8723828
 0.6217989  0.         0.6681931  0.         0.0615472  0.
 0.6659078  0.3064595  0.8230517  0.39028776]


In [13]:
# บันทึกโมเดล Autoencoder
autoencoder.save("autoencoder_model.h5")

# บันทึกโมเดล Encoder (เฉพาะส่วนที่ใช้แปลงเพลงเป็นเวกเตอร์)
encoder.save("encoder_model.h5")

# บันทึก StandardScaler ที่ใช้ Normalize ข้อมูล
import joblib
joblib.dump(scaler, "scaler.pkl")

print("✅ บันทึกโมเดลและตัวปรับมาตรฐานเรียบร้อยแล้ว!")


✅ บันทึกโมเดลและตัวปรับมาตรฐานเรียบร้อยแล้ว!


In [14]:
def recommend_similar(song_name, artist_name, df, top_n=5):
    # ค้นหาเพลงที่ชื่อเพลงตรงกัน และมีชื่อศิลปินอยู่ในลิสต์ artists
    song = df[(df["name"].str.lower() == song_name.lower()) &
              (df["artists"].apply(lambda x: artist_name.lower() in [a.lower() for a in eval(x)]))]

    if song.empty:
        return "ไม่พบเพลงนี้ใน Dataset"

    # ดึงเวกเตอร์ของเพลงที่ค้นหา
    song_vector = np.array(song["vector"].iloc[0]).reshape(1, -1)

    similarity = cosine_similarity(song_vector, np.stack(df["vector"]))

    indices = similarity.argsort()[0][::-1]

    recommended_songs = df.iloc[indices]

    # กรองเพลงที่เป็นเพลงต้นฉบับออก
    recommended_songs = recommended_songs[~((recommended_songs["name"].str.lower() == song_name.lower()) &
                                            (recommended_songs["artists"].apply(lambda x: artist_name.lower() in [a.lower() for a in eval(x)])))]

    # ลบเพลงซ้ำที่มีชื่อและศิลปินเหมือนกัน
    recommended_songs = recommended_songs.drop_duplicates(subset=["name", "artists"])

    recommended_songs = recommended_songs.head(top_n)

    return recommended_songs[["name", "artists", "album"]]

In [15]:
# ทดสอบแนะนำเพลง
recommend_similar("Pluto Projector", "Rex Orange County", df_clean)

,name,artists,album
1017415,Elephant,['Damien Rice'],9
611834,Someone To Watch Over Me,['Jan Krist'],Outpost Of The Counterculture
1069383,Before I Sleep,['Colossal Trailer Music'],Metropia
834914,Babylon Lies in Ruins,['Simon Lynge'],Deep Snow
415413,Hope Is A Thing,['Lisbeth Scott'],Rough And Steep


In [16]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from scipy.spatial import KDTree  # ✅ นำเข้า KD-Tree
import joblib
import ast

# โหลดโมเดล
autoencoder = load_model("autoencoder_model.h5", custom_objects={"mse": "mean_squared_error"})
encoder = load_model("encoder_model.h5")

# โหลดข้อมูล
df = pd.read_csv("tracks_features.csv")

# เลือกเฉพาะ Feature ที่ใช้เทรนโมเดล
features = ['danceability', 'energy', 'key', 'loudness', 'speechiness',
            'acousticness', 'instrumentalness', 'valence', 'tempo']

# ลบค่าที่เป็น NaN
df_clean = df.dropna(subset=features)

# แปลงข้อมูลเป็น numpy array
X = df_clean[features].values

# Normalize ข้อมูล
scaler = joblib.load("scaler.pkl")
X_scaled = scaler.transform(X)

# ใช้ encoder แปลงเพลงเป็นเวกเตอร์
X_encoded = encoder.predict(X_scaled)
df_clean["vector"] = list(X_encoded)

# ✅ สร้าง KD-Tree
X_encoded_array = np.array(X_encoded)
kdtree = KDTree(X_encoded_array)

# ✅ เพิ่มคอลัมน์ index เพื่อระบุตำแหน่งข้อมูลใน DataFrame
df_clean = df_clean.reset_index()

def recommend_similar(song_name, artist_name, df, top_n=5):
    # ค้นหาเพลงที่ชื่อเพลงตรงกัน และมีชื่อศิลปินอยู่ในลิสต์ artists
    song = df[(df["name"].str.lower() == song_name.lower()) &
              (df["artists"].apply(lambda x: artist_name.lower() in [a.lower() for a in eval(x)]))]

    if song.empty:
        return "ไม่พบเพลงนี้ใน Dataset"

    # ดึงเวกเตอร์ของเพลงที่ค้นหา
    song_vector = np.array(song["vector"].iloc[0]).reshape(1, -1)

    # ค้นหาเพลงด้วย KD-Tree
    distances, indices = kdtree.query(song_vector, k=top_n * 2)  # เพิ่มช่วงเผื่อกรองเพลงซ้ำ
    recommended_songs = df.iloc[indices.flatten()]

    # กรองเพลงที่เป็นเพลงต้นฉบับออก
    recommended_songs = recommended_songs[~((recommended_songs["name"].str.lower() == song_name.lower()) &
                                            (recommended_songs["artists"].apply(lambda x: artist_name.lower() in [a.lower() for a in eval(x)])))]

    # เพิ่มการเก็บเพลงที่แนะนำไปแล้วเพื่อป้องกันการแนะนำซ้ำ
    seen_songs = set()
    unique_recommendations = []

    for _, song in recommended_songs.iterrows():
        song_id = (song["name"].lower(), tuple(sorted(eval(song["artists"]))))
        if song_id not in seen_songs:
            seen_songs.add(song_id)
            unique_recommendations.append(song)
        if len(unique_recommendations) == top_n:
            break

    if not unique_recommendations:
        return "ไม่พบเพลงแนะนำที่ตรงตามเงื่อนไข"

    # แปลงกลับเป็น DataFrame เพื่อแสดงผล
    return pd.DataFrame(unique_recommendations)[["name", "artists", "album"]]


37626/37626 ━━━━━━━━━━━━━━━━━━━━ 51s 1ms/step


In [17]:
recommend_similar("Pluto Projector", "Rex Orange County", df_clean)

,name,artists,album
47417,El Dia Que Me Quieras,['Maria Volonte'],Yo Soy Maria
379983,Moment to Moment,['Kelly Eisenhour'],Seek and Find featuring Bob Mintzer
155686,"You, Sailor",['Erin McKeown'],Hundreds of Lions
611834,Someone To Watch Over Me,['Jan Krist'],Outpost Of The Counterculture
160742,Silent Night,['Wynonna'],A Classic Christmas


In [18]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.metrics.pairwise import cosine_similarity
import joblib
import ast

# โหลดโมเดล
autoencoder = load_model("autoencoder_model.h5", custom_objects={"mse": "mean_squared_error"})
encoder = load_model("encoder_model.h5")

# โหลดข้อมูล
df = pd.read_csv("tracks_features.csv")

# เลือกเฉพาะ Feature ที่ใช้เทรนโมเดล
features = ['danceability', 'energy', 'key', 'loudness', 'speechiness',
            'acousticness', 'instrumentalness', 'valence', 'tempo']

# ลบค่าที่เป็น NaN
df_clean = df.dropna(subset=features)

# แปลงข้อมูลเป็น numpy array
X = df_clean[features].values

# Normalize ข้อมูล
scaler = joblib.load("scaler.pkl")
X_scaled = scaler.transform(X)

# ใช้ encoder แปลงเพลงเป็นเวกเตอร์
X_encoded = encoder.predict(X_scaled)
df_clean["vector"] = list(X_encoded)

# ฟังก์ชันแนะนำเพลงโดยใช้ Cosine Similarity
def recommend_similar(song_name, artist_name, df, top_n=5):
    # ค้นหาเพลงที่ชื่อเพลงตรงกัน และมีชื่อศิลปินอยู่ในลิสต์ artists
    song = df[(df["name"].str.lower() == song_name.lower()) &
              (df["artists"].apply(lambda x: artist_name.lower() in [a.lower() for a in eval(x)]))]

    if song.empty:
        return "ไม่พบเพลงนี้ใน Dataset"

    # ดึงเวกเตอร์ของเพลงที่ค้นหา
    song_vector = np.array(song["vector"].iloc[0]).reshape(1, -1)

    # คำนวณความคล้ายคลึงแบบ Cosine Similarity
    similarity_scores = cosine_similarity(song_vector, np.vstack(df["vector"].values)).flatten()

    # เรียงลำดับจากความคล้ายคลึงสูงสุดไปต่ำสุด
    top_indices = np.argsort(similarity_scores)[::-1]

    # ดึงเพลงแนะนำโดยคัดกรองเพลงที่ซ้ำออก
    recommended_songs = df.iloc[top_indices]
    recommended_songs = recommended_songs[~((recommended_songs["name"].str.lower() == song_name.lower()) &
                                            (recommended_songs["artists"].apply(lambda x: artist_name.lower() in [a.lower() for a in eval(x)])))]

    # เพิ่มการเก็บเพลงที่แนะนำไปแล้วเพื่อป้องกันการแนะนำซ้ำ
    seen_songs = set()
    unique_recommendations = []

    for _, song in recommended_songs.iterrows():
        song_id = (song["name"].lower(), tuple(sorted(eval(song["artists"]))))
        if song_id not in seen_songs:
            seen_songs.add(song_id)
            unique_recommendations.append(song)
        if len(unique_recommendations) == top_n:
            break

    if not unique_recommendations:
        return "ไม่พบเพลงแนะนำที่ตรงตามเงื่อนไข"

    # แปลงกลับเป็น DataFrame เพื่อแสดงผล
    return pd.DataFrame(unique_recommendations)[["name", "artists", "album"]]


37626/37626 ━━━━━━━━━━━━━━━━━━━━ 50s 1ms/step


In [19]:
recommend_similar("Pluto Projector", "Rex Orange County", df_clean)

,name,artists,album
1017415,Elephant,['Damien Rice'],9
611834,Someone To Watch Over Me,['Jan Krist'],Outpost Of The Counterculture
1069383,Before I Sleep,['Colossal Trailer Music'],Metropia
834914,Babylon Lies in Ruins,['Simon Lynge'],Deep Snow
415413,Hope Is A Thing,['Lisbeth Scott'],Rough And Steep


In [21]:
!pip install annoy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 15.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.3-cp311-cp311-linux_x86_64.whl size=553317 sha256=eaad6a0eb748008590c3f8936331741e0318703fcfabb737ae6e11c450f48587
  Stored in directory: /root/.cache/pip/wheels/33/e5/58/0a3e34b92bedf09b4c57e37a63ff395ade6f6c1099ba59877c
Successfully built annoy


In [22]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from annoy import AnnoyIndex  # ✅ ใช้ Annoy
import joblib
import ast

# โหลดโมเดล
autoencoder = load_model("autoencoder_model.h5", custom_objects={"mse": "mean_squared_error"})
encoder = load_model("encoder_model.h5")

# โหลดข้อมูล
df = pd.read_csv("tracks_features.csv")

# เลือกเฉพาะ Feature ที่ใช้เทรนโมเดล
features = ['danceability', 'energy', 'key', 'loudness', 'speechiness',
            'acousticness', 'instrumentalness', 'valence', 'tempo']

# ลบค่าที่เป็น NaN
df_clean = df.dropna(subset=features)

# แปลงข้อมูลเป็น numpy array
X = df_clean[features].values

# Normalize ข้อมูล
scaler = joblib.load("scaler.pkl")
X_scaled = scaler.transform(X)

# ใช้ encoder แปลงเพลงเป็นเวกเตอร์
X_encoded = encoder.predict(X_scaled)
df_clean["vector"] = list(X_encoded)

# ✅ สร้าง Annoy Index
vector_dim = X_encoded.shape[1]
annoy_index = AnnoyIndex(vector_dim, 'angular')

# ✅ เพิ่มข้อมูลลงใน Annoy Index
for i, vector in enumerate(X_encoded):
    annoy_index.add_item(i, vector)

# ✅ สร้าง index เพื่อเร่งการค้นหา
annoy_index.build(10)  # 10 เป็นจำนวนต้นไม้ (trees) ที่ใช้ในการค้นหา (ยิ่งมากยิ่งแม่นยำแต่ช้าขึ้น)

# ฟังก์ชันแนะนำเพลงโดยใช้ Annoy
def recommend_similar(song_name, artist_name, df, top_n=5):
    # ค้นหาเพลงที่ชื่อเพลงตรงกัน และมีชื่อศิลปินอยู่ในลิสต์ artists
    song = df[(df["name"].str.lower() == song_name.lower()) &
              (df["artists"].apply(lambda x: artist_name.lower() in [a.lower() for a in eval(x)]))]

    if song.empty:
        return "ไม่พบเพลงนี้ใน Dataset"

    # ดึงเวกเตอร์ของเพลงที่ค้นหา
    song_index = song.index[0]
    song_vector = np.array(song["vector"].iloc[0]).tolist()

    # ✅ ค้นหาเพลงที่คล้ายกันด้วย Annoy
    similar_indices = annoy_index.get_nns_by_vector(song_vector, top_n * 2, include_distances=True)
    recommended_songs = df.iloc[similar_indices[0]]

    # กรองเพลงที่เป็นเพลงต้นฉบับออก
    recommended_songs = recommended_songs[~((recommended_songs["name"].str.lower() == song_name.lower()) &
                                            (recommended_songs["artists"].apply(lambda x: artist_name.lower() in [a.lower() for a in eval(x)])))]

    # เพิ่มการเก็บเพลงที่แนะนำไปแล้วเพื่อป้องกันการแนะนำซ้ำ
    seen_songs = set()
    unique_recommendations = []

    for _, song in recommended_songs.iterrows():
        song_id = (song["name"].lower(), tuple(sorted(eval(song["artists"]))))
        if song_id not in seen_songs:
            seen_songs.add(song_id)
            unique_recommendations.append(song)
        if len(unique_recommendations) == top_n:
            break

    if not unique_recommendations:
        return "ไม่พบเพลงแนะนำที่ตรงตามเงื่อนไข"

    # แปลงกลับเป็น DataFrame เพื่อแสดงผล
    return pd.DataFrame(unique_recommendations)[["name", "artists", "album"]]


37626/37626 ━━━━━━━━━━━━━━━━━━━━ 51s 1ms/step


In [23]:
recommend_similar("Pluto Projector", "Rex Orange County", df_clean)

,name,artists,album
611834,Someone To Watch Over Me,['Jan Krist'],Outpost Of The Counterculture
1069383,Before I Sleep,['Colossal Trailer Music'],Metropia
834914,Babylon Lies in Ruins,['Simon Lynge'],Deep Snow
214140,Mourir dans tes bras,['Ima'],A la vida !
1088798,Sweet / Sour,['Adventure Time'],"Adventure Time, Vol. 3 (Original Soundtrack)"
